In [1]:
!pip install qiskit qiskit_aer
!pip install numpy matplotlib scipy pylatexenc



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=f4ead54dcba14fb5784dfe5f15353727c7fa178392698e744f2533f46fe0b14f
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [20]:
from qiskit_aer import Aer
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
import numpy as np
from qiskit.visualization import plot_bloch_multivector

In [21]:

# draw the bloch sphere with a title
def draw_block(title, statevector):
    plot_bloch_multivector(statevector)
    plt.title(title, y=-0.01)
    plt.show()

In [22]:
#Ramsey interferometry experiment
def ramsey_experiment(phase_shift, show_bloch_sphere=False):
    # Create a quantum circuit with one qubit and one classical bit
    qc = QuantumCircuit(1, 1)

    # visualize the rotation on the bloch sphere After Initialization
    if(show_bloch_sphere):
        statevector = Aer.get_backend('statevector_simulator').run(qc).result().get_statevector()
        draw_block("Bloch Sphere after Initialization", statevector)
    # Step 1: Apply the first Hadamard gate
    qc.h(0)

    # visualize the rotation on the bloch sphere After first Hadamard
    if(show_bloch_sphere):
        statevector = Aer.get_backend('statevector_simulator').run(qc).result().get_statevector()
        draw_block("Bloch Sphere after first Hadamard", statevector)


    # Step 2: Apply the phase shift (this simulates the exposure to an external flux)
    qc.rz(phase_shift, 0)

    # Bloch Sphere after phase shift
    if(show_bloch_sphere):
        statevector = Aer.get_backend('statevector_simulator').run(qc).result().get_statevector()
        draw_block("Bloch Sphere after phase shift", statevector)

    # Step 3: Apply the second Hadamard gate
    qc.h(0)

    # Bloch Sphere after second Hadamard
    if(show_bloch_sphere):
        statevector = Aer.get_backend('statevector_simulator').run(qc).result().get_statevector()
        draw_block("Bloch Sphere after second Hadamard", statevector)
    # Step 4: Measure the qubit
    qc.measure(0, 0)

    # Run the circuit on the QASM simulator
    simulator = Aer.get_backend('qasm_simulator')
    shots = 1024
    job = simulator.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    # Calculate probability of measuring |0> and |1>
    P0 = counts.get('0', 0) / 1024
    P1 = counts.get('1', 0) / 1024

    # # graph the ramsey experiment sine wave
    # possiblePhases = np.linspace(0, 2 * np.pi, 100)
    # probabilities = 0.5 + 0.5 * np.sin(2 * possiblePhases)
    # plt.plot(possiblePhases, probabilities, label='Expected')
    # plt.xlabel('Phase Shift (radians)')
    # plt.ylabel('Probability')
    # plt.title('Ramsey Interferometry')
    return P0, P1

In [23]:

# static sensitivity of the qubit
gamma = 0.5
# known external Force at the bias point
Phi0 = 0

# run the experiment once to show the graphs

# Simulate for two different bias points: P0 = 0.5 and P0 = 1
phase_shifts = np.linspace(0, 2 * np.pi, 1000)  # Phase shift range
# linear bias points where the sinsoid graph is equal to 0.5 between 0-2pi
P0_values = [1.52, 4.72]
results = {}
# Run the experiments
P0_list = []
P1_list = []
fluxExtResults = []

#Run the algorthim once to show cases the visualization of the Bloch Sphere

In [24]:
ramsey_experiment(0.2, True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(0.9892578125, 0.0107421875)

In [25]:
for phase_shift in phase_shifts:
    P0, P1 = ramsey_experiment(phase_shift)
    P0_list.append(P0)
    P1_list.append(P1)
    # select the closest bias Point and its corresponding known external flux
    P0_value, Phi0 = (P0_values[0], 10) if abs(P0 - 0.5) < abs(P0 - 1) else (P0_values[1], 30)
    # calculate the magnitude of the external field using the bias point
    deltaP = (P1- P0_value)
    deltaPhi = 0.5*gamma/deltaP if deltaP != 0 else 0
    fluxExtResults.append(Phi0+deltaPhi)
results[1] = (P0_list, P1_list)


# Plot the results
plt.figure(figsize=(10, 6))
for P0_value in results:
    P0_list, P1_list = results[P0_value]
    label = f"P0 = {P0_value}"

    # plt.plot(phase_shifts, P0_list, label=f"P0 = {P0_value} (|0>)")
    plt.plot(fluxExtResults, P1_list, label=f"P0 = {P0_value} (|1>)", linestyle='--')

plt.title("Ramsey Interferometry: P0 = 0.5 vs P0 = 1")
plt.xlabel("External mFlux")
plt.ylabel("Probability")
plt.legend()
plt.grid(True)
plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Ramsey Interferometry: P0 = 0.5 vs P0 = 1')

Text(0.5, 0, 'External mFlux')

Text(0, 0.5, 'Probability')